In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re,nltk,json, pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
np.random.seed(42)
class color: # Text style
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
# Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# dataset path

In [ ]:
data = pd.read_csv('abp.csv',encoding='utf-8')
print(data)

In [8]:
data1 = data[['label','body']]
print(data1)

              label                                               body
0     international  জঙ্গি হটাতে যুদ্ধের ডাক দিয়েছিলেন আগেই। এ বার ...
1             state  শাসকদলের গোষ্ঠীকোন্দলের জেরে সোমবার চাষিদের নথ...
2             state  শিক্ষার সর্বস্তরে শিক্ষক-শিক্ষিকাদের হাজিরায় ব...
3          national  এখন সোশ্যাল মিডিয়া ছাড়া ভাবাই যায় না। কোনও ঘট...
4             state  তদন্তের গতিপ্রকৃতি নিয়ে আদালতে প্রশ্ন ওঠার পর ...
...             ...                                                ...
1560       national  ওই রিপোর্ট জানাল, বছরদু’য়েক আগে যে নোটবন্দি অভ...
1561          state  ফের পারদ পতন মহানগরে! হাওয়া অফিস সূত্রের খবর, ...
1562          state  এই চমকপ্রদ ঘোষণা খোদ মুখ্যমন্ত্রী মমতা বন্দ্যো...
1563          state  দাড়িভিটে গুলিবিদ্ধ হয়ে দুই ছাত্রের মৃত্যুর ঘট...
1564          state  শুধু দেহের দীর্ঘ আয়ুষ্কাল কোন স্রষ্টারই বা কাঙ...

[1565 rows x 2 columns]


In [ ]:
# Plot the Class distribution
sns.set(font_scale=1.4)
data['label'].value_counts().plot(kind='barh', figsize=(8, 6))
plt.xlabel("Number of Articles", labelpad=12)
plt.ylabel("label", labelpad=12)
plt.yticks(rotation = 45)
plt.title("Dataset Distribution", y=1.02);

In [ ]:
sample_data = [20,50,100,200,300,350,400,450,500,600,650,700,750,800]
for i in sample_data:
      print('body:\n',data.body[i],'Category:-- ',data.label[i],'\n')  

In [ ]:
 news = re.sub('[^\u0980-\u09FF]',' ',str(data.body[i]))

In [ ]:
print(news)

In [11]:
#########------------------------
#############----------------------------------this one is not working
# Cleaning Data [Remove unncessary symbols]
def cleaning_documents(articles):
      '''
      This function will clean the news articles by removing punctuation marks and stopwords.
      '''
      news = re.sub('\n',' ',str(articles))  
     #removing unnecessary punctuation
     #link: https://www.google.com/search?q=removing+puntuation+regular+expression+python&rlz=1C1CHWL_enIN894IN894&oq=removing+puntuation+regular+expression+python&aqs=chrome..69i57j33i22i29i30l2.13383j1j7&sourceid=chrome&ie=UTF-8#kpvalbx=_4H-8Y86lEKqOxc8PvrSx-A4_42
      news = re.sub('[^\u0980-\u09FF]',' ',str(news))        
      # stopwords removal
      stp = open('stopwords.txt','r',encoding='utf-8').read().split()
      result = news.split()
      news = [word.strip() for word in result if word not in stp ]
      news =" ".join(news)
      return news


# Apply the function into the dataframe
data1['cleaned'] = data1['body'].apply(cleaning_documents)  

<ipython-input-11-933c373ebc73>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['cleaned'] = data1['body'].apply(cleaning_documents)


In [25]:
print(len(data))

1565


In [12]:
# Cleaning Data [Remove unncessary symbols]
def cleaning_documents(articles):
      '''
      This function will clean the news articles by removing punctuation marks and stopwords.
      '''
      news = re.sub('\n',' ',str(articles))  
     #removing unnecessary punctuation
     #link: https://www.google.com/search?q=removing+puntuation+regular+expression+python&rlz=1C1CHWL_enIN894IN894&oq=removing+puntuation+regular+expression+python&aqs=chrome..69i57j33i22i29i30l2.13383j1j7&sourceid=chrome&ie=UTF-8#kpvalbx=_4H-8Y86lEKqOxc8PvrSx-A4_42
      news = re.sub('[^\u0980-\u09FF]',' ',str(news))        
      # stopwords removal
      stp = open('stopwords.txt','r',encoding='utf-8').read().split()
      result = news.split()
      news = [word.strip() for word in result if word not in stp ]
      news =" ".join(news)
      return news
#cleaned=[]

for i in range(len(data)):
    data1.cleaned[i]=cleaning_documents(data1.body[i])

C:\Users\somrita\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print('after change')
for i in sample_data:
      print('body:\n',data1.cleaned[i],'Category:-- ',data1.label[i],'\n')  

In [15]:
# Calculate the Length of each Document
data1['Length'] = data1.cleaned.apply(lambda x:len(x.split()))

In [16]:
'''
################check1------->remove 0 length record
'''
print(data1['Length'])

for i in range(len(data1)):
    if data1.Length[i]<25:
        print(i,data1.Length[i],data1.cleaned[i])

0       163
1        67
2       266
3       161
4       174
       ... 
1560    128
1561    174
1562    463
1563    161
1564    388
Name: Length, Length: 1565, dtype: int64
23 0 
343 0 
433 23 পোস্টার লঞ্চে অভিনবত্য দেখাল তিন টিম পরিচালক ঋভু দাশগুপ্ত তিন টিম সমর্থদের গিয়েছিলেন অভিনব প্রতিযোগিতা প্রতিযোগিতার সাকসেস পার্টি গ্র্যান্ড পার্টিতে চোখ রাখুন গ্যালারিতে
825 21 ভাবুন রাস্তা দিয়ে হাঁটছেন হঠাত্ই পাশ দিয়ে জল রাস্তায় ভরে জলের উচ্চতা সমান হ্যাঁ রকমই রাস্তা রয়েছে ফ্রান্সে জেনে নিন গ্যালারিতে
967 24 মির্জা পরিবারে বিয়ের সানাই বাজল বিয়ে সানিয়া মির্জার বোন আনম মির্জা সঙ্গীত অনুষ্ঠানে হাজির টাউনের তাবড় তারকারা এক নজরে নিন আনমের বিয়ের সঙ্গীনুষ্ঠানের ঝলক
1069 0 
1227 24 অসম বয়স অসম বন্ধুত্ব এক প্রান্তে জ্যাক রাসেল কুকুর অপর প্রান্তে চিতাবাঘ খাওয়া ঘুম সবই একসঙ্গে অপরের গায়ে ওঠে খেলাও কেমন বন্ধুত্ব দেখুন ভিডিও
1509 23 ম্যাক্সিম ম্যাগাজিনের সুপার মডেলকে চিনতে পারছেন নাহ হলিউডি অভিনেত্রীর কথা বলছি ভাল দেখলেই বুঝবেন বলিউডের পর্দা কাঁপানো নায়িকা চেনা চিনে নিন সঙ্গের গ্যালারি


In [ ]:
import matplotlib
matplotlib.rc_file_defaults()
frequency = dict()
for i in data1.Length:
    frequency[i] = frequency.get(i, 0)+1

plt.figure(figsize = (6,4))
plt.bar(frequency.keys(), frequency.values(), color=(0.2, 0.4, 0.6, 0.6))
plt.xlim(0, 800)
# in this notbook color is not working but it should work.
plt.xlabel('Length of the Documents')
plt.ylabel('Frequency')
plt.title('Length-Frequency Distribution')
#plt.savefig(path+'len_dist.png',dpi = 1000,bbox_inches ='tight')
plt.show()  
print(f"Maximum Length of a Document: {max(data1.Length)}")
print(f"Minimum Length of a Document: {min(data1.Length)}")
print(f"Average Length of a Document: {round(np.mean(data1.Length),0)}")


In [18]:
 class_label = [k for k,v in data1.label.value_counts().to_dict().items()]

In [19]:
print( class_label)

['national', 'state', 'international', 'entertainment', 'kolkata', 'sport']


In [20]:
'''
#######################check2------->stopwords are not removed properly---diye/hoye

'''
def data_summary(dataset):
  '''
  This function will print the summary of the dataset

  Args:
  dataset: cleaned texts

  returns:
  documents: Number of document per class
  words: Number of words per class
  u_words: Number of unique words per class
  class_label: name of the categories
  '''
  documents = []
  words = []
  u_words = []
  #total_u_word = [word.strip().lower() for t in list(dataset.cleaned) for word in t.strip().split()]
  # find class names
  class_label = [k for k,v in data1.label.value_counts().to_dict().items()]
  for label in class_label: 
    word_list = [word.strip().lower() for t in list(data1[data1.label==label].cleaned) for word in t.strip().split()]
    counts = dict()
    for word in word_list:
      counts[word] = counts.get(word, 0)+1
    # sort the dictionary of word list  
    ordered = sorted(counts.items(), key= lambda item: item[1],reverse = True)
    # Documents per class
    documents.append(len(list(data1[data1.label==label].cleaned)))
    # Total Word per class
    words.append(len(word_list))
    # Unique words per class 
    u_words.append(len(np.unique(word_list)))
    print("\nClass Name : ",label)
    print("Number of Documents:{}".format(len(list(data1[data1.label==label].cleaned))))  
    print("Number of Words:{}".format(len(word_list))) 
    print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
    print("Most Frequent Words:\n")
    for k,v in ordered[:10]:
      print("{}\t{}".format(k,v))
  return documents,words,u_words,class_label

# pass the whole dataset into the function

documents,words,u_words,class_names = data_summary(data1)
       


Class Name :  national
Number of Documents:665
Number of Words:138202
Number of Unique Words:23475
Most Frequent Words:

এক	933
হয়েছে	931
মোদী	735
কথা	650
বিজেপি	628
বিরুদ্ধে	485
দাবি	480
প্রধানমন্ত্রী	472
কংগ্রেস	449
হয়ে	433

Class Name :  state
Number of Documents:436
Number of Words:108041
Number of Unique Words:20765
Most Frequent Words:

এক	798
হয়েছে	791
রাজ্য	491
টাকা	433
পুলিশ	427
অভিযোগ	420
কথা	408
দাবি	377
রাজ্যের	355
হয়ে	354

Class Name :  international
Number of Documents:227
Number of Words:40799
Number of Unique Words:11816
Most Frequent Words:

এক	353
হয়েছে	242
মার্কিন	186
কথা	167
দেশের	158
হয়ে	158
ভারত	152
ভারতের	144
দাবি	141
দিয়ে	133

Class Name :  entertainment
Number of Documents:192
Number of Words:23878
Number of Unique Words:8506
Most Frequent Words:

ছবি	215
এক	171
ছবির	126
একটা	111
কথা	108
হয়েছে	105
ছবিতে	104
হয়ে	93
অভিনয়	88
সোশ্যাল	68

Class Name :  kolkata
Number of Documents:25
Number of Words:5334
Number of Unique Words:2540
Most Frequent Words:

কলকাতা	41
এ

In [21]:
data_matrix = pd.DataFrame({'Total Documents':documents,
                            'Total Words':words,
                            'Unique Words':u_words,
                            'Class Names':class_names})
data_matrix

,Total Documents,Total Words,Unique Words,Class Names
0,665,138202,23475,national
1,436,108041,20765,state
2,227,40799,11816,international
3,192,23878,8506,entertainment
4,25,5334,2540,kolkata
5,20,4538,2347,sport


In [ ]:

sns.set()
plt.figure(figsize =(10, 8))
ax = plt.subplot()
sns.heatmap(data_matrix.iloc[:,0:3], annot=True,fmt="d", linewidths=0.5,linecolor = 'Black',cmap = "YlGnBu",ax = ax)
# labels, title and ticks
ax.set_ylabel('Class Names') 
ax.set_title('Data Statistics')
y_label = ["Total Documents", "Total Words", "Unique Words"] 
ax.xaxis.set_ticklabels(y_label, rotation=45); ax.yaxis.set_ticklabels(class_names, rotation=45);
ax.xaxis.tick_top()

In [24]:
 ################# Label Encoding Function ##################
                   #===========================================================
def label_encoding(dataset,bool):
  '''
  This function will convert categorical class names into a numeric label

  Args:
  dataset: A dataset
  bool : a boolean (True or False)

  returns:
  labels: encoded labels 
  '''
  le = LabelEncoder()
  le.fit(data1.label)
  encoded_labels = le.transform(data1.label)
  labels = np.array(encoded_labels) # Converting into numpy array
  class_names =le.classes_ ## Define the class names again
  if bool == True:
    print(color.BOLD+"\n\t\t\t===== Label Encoding ====="+color.END,"\nClass Names:-->",le.classes_)
  return labels  

 ################# Dataset Splitting Function ###############
                   #=========================================================== 

def dataset_split(news,category):
  '''
  This function will split the dataset into Train-Test-Validation set

  Args:
  news: encoded texts
  category: class names

  returns:
  X_train: Encoded Training News  
  X_valid: Encoded Validation News
  X_test:  Encoded Test news
  y_train: Encoded Training labels
  y_valid: Encoded Validation labels
  y_test:  Encoded Test labels
  '''

  X,X_test,y,y_test = train_test_split(news,category,train_size = 0.9,
                                                  test_size = 0.1,random_state =0)
  X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size = 0.8,
                                                  test_size = 0.2,random_state =0)
  print(color.BOLD+"Feature Size :======>"+color.END,X_train.shape[1])
  print(color.BOLD+"\nDataset Distribution:\n"+color.END)
  print("\tSet Name","\t\tSize")
  print("\t========\t\t======")

  print("\tFull\t\t\t",news.shape[0],
        "\n\tTraining\t\t",X_train.shape[0],
        "\n\tTest\t\t\t",X_test.shape[0],
        "\n\tValidation\t\t",X_valid.shape[0])
  
  return X_train,X_valid,X_test,y_train,y_valid,y_test


In [26]:

def encoded_texts(dataset,padding_length,max_words):

  '''
  This function will encoded the text into a sequence of numbers

  Args:
  dataset: a dataset
  padding_length: maximum length of a encoded texte
  max_words : Number of words 

  returns:
  corpus: Number of encoded texts
  labels: encoded labels
  '''
  
  tokenizer = Tokenizer(num_words = max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n-', 
                      split=' ', char_level=False, oov_token='', document_count=0)
  
 

 # Fit the tokenizer
  tokenizer.fit_on_texts(dataset.cleaned)
  #============================== Tokenizer Info =================================
  (word_counts,word_docs,word_index,document_count) = (tokenizer.word_counts,
                                                       tokenizer.word_docs,
                                                       tokenizer.word_index,
                                                       tokenizer.document_count)
  def tokenizer_info(mylist,bool):
    ordered = sorted(mylist.items(), key= lambda item: item[1],reverse = bool)
    for w,c in ordered[:10]:
      print(w,"\t",c)
  #=============================== Print all the information =========================
  print(color.BOLD+"\t\t\t====== Tokenizer Info ======"+color.END)   
  print("Words --> Counts:")
  tokenizer_info(word_counts,bool =True )
  print("\nWords --> Documents:")
  tokenizer_info(word_docs,bool =True )
  print("\nWords --> Index:")
  tokenizer_info(word_index,bool =True )    
  print("\nTotal Documents -->",document_count)

  #=========================== Convert string into list of integer indices =================
  sequences = tokenizer.texts_to_sequences(dataset.cleaned)
  word_index = tokenizer.word_index
  print(color.BOLD+"\n\t\t\t====== Encoded Sequences ======"+color.END,"\nFound {} unique tokens".format(len(word_index)))  
  print(dataset.cleaned[10],"\n",sequences[10]) 

  #==================================== Pad Sequences ============================== 
  corpus = keras.preprocessing.sequence.pad_sequences(sequences, value=0.0,
                                                      padding='post', maxlen= padding_length)
  print("\n\t\t\t====== Paded Sequences ======\n",dataset.cleaned[10],"\n",corpus[10])   

  #=================================     Label Encoding ================================
  labels = label_encoding(dataset,True)

   # save the tokenizer into a pickle file
  with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

  return corpus,labels
  
#call the function
num_words = 5000
corpus,labels = encoded_texts(data1,
                                300,num_words)
print("\nShape of Encoded Corpus =====>",corpus.shape)

			====== Tokenizer Info ======
Words --> Counts:
এক 	 2331
হয়েছে 	 2108
কথা 	 1363
হয়ে 	 1087
দাবি 	 1029
পুলিশ 	 968
অভিযোগ 	 914
দিয়ে 	 888
মোদী 	 873
রয়েছে 	 859

Words --> Documents:
এক 	 1052
হয়েছে 	 953
কথা 	 736
হয়ে 	 653
দিয়ে 	 558
দাবি 	 547
রয়েছে 	 547
গত 	 526
যায় 	 503
খবর 	 489

Words --> Index:
বলেওছিলেন 	 42407
প্রয়াণকেই 	 42406
জিনিসটার 	 42405
বিনয়মাত্র 	 42404
অমলকান্তি 	 42403
কল্পনাপ্রবণ 	 42402
সহনীয় 	 42401
অভিঘাত 	 42400
পরিবারমনস্ক 	 42399
কোমলহৃদয় 	 42398

Total Documents --> 1565

			====== Encoded Sequences ====== 
Found 42407 unique tokens
বুলন্দশহরের হত্যাকাণ্ডকে বিজেপি সভাপতি অমিত শাহ দুর্ভাগ্যজনক রাজস্থানে বিধানসভা ভোটের প্রচারের শেষ দিনে প্রধানমন্ত্রী নরেন্দ্র মোদী এক ঢিলে পাখি মারতে চেয়েছেন এক বুলন্দশহরের রাজনৈতিক মেরুকরণের সরাসরি ফায়দা রাজস্থানে চাইছেন উত্তরপ্রদেশের ঘটনাকে আইনশৃঙ্খলার অবনতি আখ্যা দিয়ে দোষী সাব্যস্ত মুখ্যমন্ত্রী যোগী আদিত্যনাথকে উত্তরপ্রদেশে বিধানসভা ভোট লোকসভার বাবরি মসজিদ ধ্বংসের তারিখ ৬ ডিসেম্বরের পরের দিনই রাজস্থানে ভোট গো বলয়ের রা

In [27]:
X_train,X_valid,X_test,y_train,y_valid,y_test = dataset_split(corpus,labels)

Feature Size :======> 300

Dataset Distribution:

	Set Name 		Size
	========		======
	Full			 1565 
	Training		 1126 
	Test			 157 
	Validation		 282


In [ ]:



embedding_dimension = 128
input_length = 300
vocab_size = 5000
num_classes = 12
batch_size = 64
num_epochs = 10

accuracy_threshold = 0.97


class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = "Model.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 

                            ###############################     CNN network architecture ################
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(vocab_size, embedding_dimension, input_length = input_length),
tf.keras.layers.Conv1D(128, 5, activation= 'relu'),
tf.keras.layers.MaxPooling1D(5),
tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True,dropout = 0.2)),
tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True,dropout = 0.2)),
tf.keras.layers.Dense(28, activation='relu'),
tf.keras.layers.Dense(14, activation='relu'),
keras.layers.Flatten(),
tf.keras.layers.Dense(num_classes , activation='softmax')])

model.summary()

model.compile(optimizer=keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
   
history = model.fit(X_train,
    y_train,
    epochs=num_epochs,
    batch_size = batch_size,
    validation_data=(X_valid, y_valid),
    verbose =1)
    #callbacks = callback_list)

In [29]:
# save the model
tf.keras.models.save_model(model,'Document.h5', save_format='h5')
#model.save(path+'Document_Categorization.h5')

In [ ]:

loss_values = history.history['loss']
val_loss_values = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)
sns.set_style('darkgrid')
                  #=========================================================
                  ############### Epochs vs Validation Accuracy Plot ##########
                  #==========================================================

plt.plot(epochs, acc, color='midnightblue', linewidth = 2, 
          marker='o', markersize=8,label='Training Accuracy')
plt.plot(epochs, val_acc, color='darkred', linewidth = 2, 
          marker='o', markersize=8,label='Training Accuracy')
plt.title('Training and validation accuracy',fontsize=12,fontweight='bold')
plt.xlabel('Epochs',fontsize=12,fontweight='bold')
plt.ylabel('Accuracy',fontsize=12,fontweight='bold')
plt.legend(['Training Accuracy','Validation Accuracy'])

In [32]:
# Load the saved model
model = load_model('Document.h5')

In [35]:
class_names =  ['entertainment' 'international' 'kolkata' 'national' 'sport' 'state']
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)
report = pd.DataFrame(classification_report(y_true = y_test, y_pred = y_pred, output_dict=True)).transpose()
report = report.rename(index={'0':'entertainment','1':'international','2':'kolkata','3':'national','4':'Education','5':'sport','6':'state'})
report[['precision','recall','f1-score']]=report[['precision','recall','f1-score']].apply(lambda x: round(x*100,2))
report

5/5 [==============================] - 1s 25ms/step


C:\Users\somrita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\somrita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\somrita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
entertainment,85.71,57.14,68.57,21.000000
international,39.29,40.74,40.00,27.000000
kolkata,0.00,0.00,0.00,5.000000
national,66.20,92.16,77.05,51.000000
Education,0.00,0.00,0.00,4.000000
sport,86.05,75.51,80.43,49.000000
accuracy,68.15,68.15,68.15,0.681529
macro avg,46.21,44.26,44.34,157.000000
weighted avg,66.58,68.15,66.18,157.000000


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

cm = confusion_matrix(y_test, y_pred) 

class_names = ['entertainment' 'international' 'kolkata' 'national' 'sport' 'state']
# Transform to df for easier plotting
cm_df = pd.DataFrame(cm,
                     index = class_names, 
                     columns = class_names)

plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True,cmap="Greens", fmt='g')
plt.title('CNN-BiLSTM \nAccuracy: {0:.2f}'.format(accuracy_score(y_test, y_pred)*100))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.show()

In [ ]:
sample_news = 'শিয়ালদহ স্টেশনে দাঁড়িয়ে থাকা লোকাল ট্রেনের ছাদ থেকে উদ্ধার হল এক অজ্ঞাতপরিচয় ব্যক্তির দেহ। ঘটনাটি ঘিরে চাঞ্চল্য ছড়ায় শিয়ালদহ দক্ষিণ শাখার স্টেশন চত্বরে। রেল পুলিশের ধারণা, সম্ভবত বিদ্যুৎস্পৃষ্ট হয়েই মৃত্যু হয়েছে ওই ব্যক্তির। এ দিন সকালে আপ ক্যানিং লোকাল শিয়ালদহ স্টেশনে এসে পৌঁছয়। তখনই ট্রেনের ছাদে এক ব্যক্তির দেহ নজরে আসে যাত্রী এবং আরপিএফ কর্মীদের। এর পরে ওই দেহটি উদ্ধার করা হয়। প্রাথমিকভাবে পুলিশকর্মীদের অনুমান, মানসিক ভারসাম্যহীন ওই ব্যক্তি কোনওভাবে ট্রেনে ছাদে উঠে পড়েছিলেন। এর পরে ট্রেনের ছাদের প্যান্টোগ্রাফের কাছে চলে যান '
cleaned_news = cleaning_documents(sample_news)
class_names = ['entertainment' 'international' 'kolkata' 'national' 'sport' 'state']

#print(cleaned_news)
# load the saved tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
#print(len(loaded_tokenizer.word_index))
seq= loaded_tokenizer.texts_to_sequences([cleaned_news])
padded = pad_sequences(seq, value=0.0,padding='post', maxlen= 500 )
pred = model.predict(padded)
#print(pred) 
class_names[np.argmax(pred)]   

In [43]:
str1=sample_news = ' নগদহীন লেনদেনের লক্ষ্যে এবার আরও একটা গুরুত্বপূর্ণ সিদ্ধান্ত নিল কেন্দ্র। কর্মীদের আর নগদে বেতন দেওয়া যাবে না। চেক বা ব্যাঙ্ক অ্যাকাউন্টের মাধ্যমে দেওয়া হবে বেতন। বুধবার কেন্দ্রীয় মন্ত্রিসভার বৈঠকে এমনই সিদ্ধান্ত গৃহীত হয়েছে। নোটবাতিলের পরেই জানা গিয়েছিল, নগদহীন লেনদেনকে উৎসাহিত করতে আরও নানা সিদ্ধান্ত নেবে মোদী সরকার। বুধবার কেন্দ্রীয় মন্ত্রিসভার বৈঠকে যে গুরুত্বপূর্ণ কোনও সিদ্ধান্ত হতে পারে সেটাও জানা ছিল। সেই মতো এদিন মন্ত্রিসভার বৈঠকে কর্মচারীদের নগদে বেতন দেওয়ার ক্ষেত্রে নিষেধাজ্ঞা জারির অর্ডিন্যান্সে সিলমোহর দেওয়া হয়েছে। এর ফলে চেকে বা সরাসরি কর্মচারীদের ব্যাঙ্ক অ্যাকাউন্টে বেতন দিতে হব ে। েখ্য, এ সংক্রান্ত বিল গত ১৫ ডিসেম্বর লোকসভায় পেশ করা হয়। এই বিল আগামী বাজেট অধিবেশনে পাস করানো হতে পারে। অর্থাৎ বিল পাস করাতে এখনও দু’মাস সময় লেগে যাবে। তাই এ বিষয়ে এখন অর্ডিন্যান্স জারি করা হল। পরে তা সংসদে পাস করানো হবে। নিয়ম অনুযায়ী, অর্ডিন্যান্স ছ’মাসের জন্য বৈধ থাকে। এর মধ্যে তা সংসদে অনুমোদন করতে হয়। বেতন প্রদান (সংশোধনী) বিল ২০১৬-তে মূল আইনের ৬ নম্বর ধারায় সংশোধনের প্রস্তাব করা হয়েছে, যাতে নিয়োগকারী কর্মচারীদের বেতন চেক বা ইলেকট্রনিক পদ্ধতিতে সরাসরি ব্যাঙ্ক অ্যাকাউন্টে পাঠাতে পারেন।'
len(str1)

1098